In [1]:
import numpy as np
from pyrecode.params import InputParams, InitParams
from pyrecode.recode_server import ReCoDeServer
from pyrecode.recode_reader import ReCoDeReader, merge_parts

## Converting from Numpy dense array to ReCoDe format

1. Initialise data and the desired parameters.
- Replace the random integer array in `data` with actual Numpy array data with the dimensions (nz, nx, ny).
- Change parameters in `InitParams` accordingly:

`InitParams(mode, output_directory, image_filename='', directory_path='', calibration_filename='', params_filename='', validation_frame_gap=-1, log_filename='recode.log', run_name='run', verbosity=0, use_c=False, max_count=-1, chunk_time_in_sec=0)`

__Note:__ If `mode` in `InitParams` is changed to 'stream', `mode` in `ReCoDeServer` should be changed as well.

In [2]:
tag = 'test'
data = np.random.randint(4096, size=(9, 512, 512), dtype=np.uint16)
shape = np.shape(data)

calib_frame = np.zeros((shape[1], shape[2]), dtype=np.uint16)

init_params = InitParams('batch', '../convert', image_filename='test_data',
                             validation_frame_gap=2, log_filename='../convert/recode.log',
                             run_name=tag, verbosity=0, use_c=False)

input_params = InputParams()
input_params.load('../convert/recode_params_' + tag + '.txt')
input_params.nx = shape[1]
input_params.ny = shape[2]
input_params.nz = shape[0]
input_params.source_data_type = 0 # unsigned int
input_params.target_data_type = 0 # unsigned int

server = ReCoDeServer('batch')
run_metrics = server.run(init_params, input_params=input_params, dark_data=calib_frame, data=data)

# Merge intermediate files
merge_parts('../convert', 'test_data.rc1', 3)

Node 0 NodeReady at port  118534 
Ready at portNode 18535 
2 Ready at port 18536
(INFO) 07/09/2021-12:22:39 Logger : Logger is up and running  [Session ID:10053]
(INFO) 07/09/2021-12:22:39 Node 2 : Node 2 Waiting...  [Session ID:10053]
(INFO) 07/09/2021-12:22:39 Head Node : Welcome to this session  [Session ID:10053]
Node 0  Received:
{'session_id': 10053, 'type': 3, 'message': 'start', 'descriptive_message': '', 'source_process_id': 8514, 'target_process_id': 0, 'mapped_data': {'req_id': 1}}
ReCoDe Header
-------------Node
 uid1  = Received: 
158966344846346{'session_id': 10053, 'type': 3, 'message': 'start', 'descriptive_message': '', 'source_process_id': 8514, 'target_process_id': 1, 'mapped_data': {'req_id': 1}}

version_major ReCoDe Header=
Node ------------- 0
2
uid version_minor  Received: =
= {'session_id': 10053, 'type': 3, 'message': 'start', 'descriptive_message': '', 'source_process_id': 8514, 'target_process_id': 2, 'mapped_data': {'req_id': 1}} 158966344846346
2
version_m

(INFO) 07/09/2021-12:22:41 Node 2 : Processed 3 frames in 0:00:00.878515  [Session ID:10053]
(INFO) 07/09/2021-12:22:41 Node 2 : Estimated dose rate = 6.103515625e-05  [Session ID:10053]
(INFO) 07/09/2021-12:22:41 Node 2 : Node 2 Waiting...  [Session ID:10053]
(INFO) 07/09/2021-12:22:41 Node 0 : Processed 3 frames in 0:00:00.905210  [Session ID:10053]
(INFO) 07/09/2021-12:22:41 Node 0 : Estimated dose rate = 6.103515625e-05  [Session ID:10053]
(INFO) 07/09/2021-12:22:41 Node 0 : Node 0 Waiting...  [Session ID:10053]
(INFO) 07/09/2021-12:22:41 Node 1 : Processed 3 frames in 0:00:00.906073  [Session ID:10053]
(INFO) 07/09/2021-12:22:41 Node 1 : Estimated dose rate = 6.103515625e-05  [Session ID:10053]
(INFO) 07/09/2021-12:22:41 Node 1 : Node 1 Waiting...  [Session ID:10053]
Node 0  Received:
{'session_id': 10053, 'type': 3, 'message': 'close', 'descriptive_message': '', 'source_process_id': 8514, 'target_process_id': 0, 'mapped_data': {'req_id': 3}}
(INFO) 07/09/2021-12:22:41 Node 0 : Cl

---

## Converting from ReCoDe format to Numpy dense array

1. ReCoDe file can be read using `ReCoDeReader`, which fetches frames in the form of a sparse COO matrix.

Alternatively, to fetch a batch of data instead of a single frame, the `ReaderNode` class from `ReCoDe_Live_View_example.ipynb` may be used.

In [3]:
recode_file_name = '../convert/test_data.rc1'
reader = ReCoDeReader(recode_file_name, is_intermediate=False)
reader.open()

ReCoDe Header
-------------
uid = 158966344846346
version_major = 0
version_minor = 2
is_intermediate = 1
reduction_level = 1
rc_operation_mode = 1
is_bit_packed = 1
target_bit_depth = 12
nx = 512
ny = 512
nz = 9
frame_metadata_size = 0
num_non_standard_frame_metadata = 0
L2_statistics = 0
L4_centroiding = 0
compression_scheme = 0
compression_level = 1
source_file_type = 0
source_header_length = 0
source_header_position = 0
source_file_name = test_data                                                                                           
calibration_file_name =                                                                                                     
calibration_threshold_epsilon = 0
has_calibration_data = 1
frame_offset = 0
calibration_frame_offset = 0
num_calibration_frames = 1
source_bit_depth = 12
source_dtype = 0
target_dtype = 0
checksum = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
futures = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [4]:
#Fetch one frame
frame = reader.get_next_frame()

if frame == None:
    print("Reached end of file - no remaining frames to view.")
else:
    frame_id = list(frame.keys())[0]
    data = frame[frame_id]['data'] # COO matrix
    dense_array = data.todense() # Convert to Numpy dense array
    
print(dense_array)

[[2938 1748  373 ... 3781 3105 3453]
 [2882 1555 2861 ... 4052 2818 3701]
 [1274 3665  502 ... 3626 3866   94]
 ...
 [2277 1372 1728 ... 1682  128 3896]
 [ 870 2359  125 ...  174 2619 4024]
 [3845 3944 1243 ...  403 2015 3173]]
